In [1]:
"""
Imports
"""
from functions import *
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

In [2]:
def poly_features(X, degree):
    matrix = np.zeros((X.shape[0], degree))
    for i in range(X.shape[0]):
        for j in range(0, degree):
            matrix[i, j] = X[i,] ** (j + 1)
    # print(matrix)
    return matrix

def linear_fit(X, Y):
    if type(X.T.dot(X)) == type(np.float64(0.1)):
        return 1/(X.T.dot(X)) * (X.T).dot(Y)
    else:
        return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)
    # return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y) if type(X.T.dot(X)) != type(1) else 1/(X.T.dot(X)) * (X.T).dot(Y)

def offset(a, X, Y):
    temp = np.tile(np.array(np.sum(X)/X.shape[0]), a.shape)
    if type(a) == type(np.float64(0.1)):
        return np.sum(Y)/Y.shape[0] - temp * a
    else:
        return np.sum(Y)/Y.shape[0] - temp.T.dot(a)

def cross_val(X, Y, degree, off=False, folds=10):
    train_size = int(X.shape[0] * 0.8)
    errors = []
    r_squareds = []
    row_index = np.array([i for i in range(X.shape[0])])
    for fold in range(folds):
        rows = np.random.choice(X.shape[0], train_size, replace=False)
        not_rows = []
        for i in row_index:
            if i not in rows:
                not_rows.append(i)
        train_data = X[rows]
        train_label = Y[rows]
        test_data = X[np.array(not_rows)]
        test_label = Y[np.array(not_rows)]
        X_poly = poly_features(train_data, degree)
        test_poly = poly_features(test_data, degree)
        a = linear_fit(X_poly, train_label)
        if off:
            b = offset(a, X_poly, Y)
        else:
            b = 0
        Y_ = test_poly.dot(a) + b
        errors.append(calc_error(test_label, Y_))
        r_squareds.append(r_squared(test_label, Y_))

    return sum(errors)/len(errors), sum(r_squareds)/len(r_squareds)

def calc_error(Y, Y_hat):
    return np.sum((Y-Y_hat)**2/Y.shape[0])

def r_squared(Y, Y_hat):
    return 1 - np.sum((Y-Y_hat) ** 2)/np.sum((Y-np.sum(Y)/Y.shape[0])**2)

def bic(Y, Y_hat, k):
    temp = Y - Y_hat
    n = Y.shape[0]
    return n * np.log(temp.dot(temp.T)/n) + k * np.log(n)

In [3]:
"""
Read files
df_users: for Outdoor
df_users_split: for Indoor
"""
df = pd.read_csv("Results/1_HourAverage.csv")
df['PT DateTime'] = pd.to_datetime(df['PT DateTime'])
df_users = pd.read_excel("D:/UW/AeroSpec - Sensor Network/2020 Wildfire/Data/Device Locations and Users.xlsx",
                             engine='openpyxl')
df_users = df_users[df_users['In'].notna()].reset_index(drop=True)
df_users_split = pd.read_excel(
    "D:/UW/AeroSpec - Sensor Network/2020 Wildfire/Data/Device Locations and Users Split.xlsx", engine='openpyxl')
df_users_split = df_users_split[df_users_split['In'].notna()].reset_index(drop=True)
df_10min = pd.read_csv("Results/2_10MinAverage.csv")
df_10min['PT DateTime'] = pd.to_datetime(df_10min['PT DateTime'])
df_10sec = pd.read_csv("Results/3_10SecAverage.csv")
df_10sec['PT DateTime'] = pd.to_datetime(df_10sec['PT DateTime'])
public_sensors = ["Lake Forest Park", "Seattle 10th & Weller"]
df_out = average(df, df_users, "Out", "PM2.5_Env",end="9/21/2020 0:00")
df_in = average(df, df_users_split, "In", "PM2.5_Env", breakout=True, end="9/21/2020 0:00")
df_out10min = average(df_10min, df_users, "Out", "PM2.5_Env", end="9/21/2020 0:00")
df_in10min = average(df_10min, df_users_split, "In", "PM2.5_Env", breakout=True, end="9/21/2020 0:00")
dfs = []
for device in public_sensors:
    df_temp = df[df["Device Name"] == device] \
        [(df["PT DateTime"] >= "9/10/2020 0:00") & (df["PT DateTime"] <= "9/21/2020 0:00")] \
        [["PT DateTime", "PM2.5_Std"]]
    df_temp.rename(columns={"PM2.5_Std": device}, inplace=True)
    dfs.append(df_temp)
df_pub = pd.merge(dfs[0], dfs[1], on="PT DateTime", how="left")
cols = list(df_pub.columns)
cols.pop(0)
df_pub["Average"] = df_pub[cols].mean(axis=1)

6    Beta-08
Name: Out, dtype: object
6    Beta-13
Name: In, dtype: object
6    Beta-08
Name: Out, dtype: object
6    Beta-13
Name: In, dtype: object


D:\pythonProject\AeroSpecProject\functions.py:790: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = df[df["Device Name"] == df_users[df_users["User"] == "MEB"][in_or_out].values[0]] \
D:\pythonProject\AeroSpecProject\functions.py:799: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = pd.merge(df_temp, df[df["Device Name"] == df_out_users[i]] \
<ipython-input-3-bb40ea7c5dec>:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_temp = df[df["Device Name"] == device] \


In [16]:
df_pub
df_pub["Lake Forest Park"].corr(df_pub["Seattle 10th & Weller"])

0.9663463138547197

In [4]:
df_out1 = df_out[["PT DateTime", "Beta-08", "Beta-16", "Beta-17", "Beta-03", "Breakout-08", "Breakout-06"]]
df_pub1 = df_pub[["PT DateTime", "Average"]]
df_pub1.rename(columns={"Average": "Reference Average"}, inplace=True)
df_test = pd.merge(df_out1, df_pub1, on="PT DateTime").dropna()
df_test

C:\Users\a1295\anaconda3\envs\AeroSpecProject\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,PT DateTime,Beta-08,Beta-16,Beta-17,Beta-03,Breakout-08,Breakout-06,Reference Average
7,2020-09-10 13:00:00,54.094444,52.225000,51.569444,43.948052,46.850000,42.870000,32.55
8,2020-09-10 14:00:00,54.677778,62.163889,55.711111,48.044379,48.970833,43.640000,35.05
9,2020-09-10 15:00:00,60.744444,98.450000,61.961111,48.099707,50.495833,47.629630,34.90
10,2020-09-10 16:00:00,57.036111,59.302778,62.652778,57.267267,48.295833,46.906040,38.95
11,2020-09-10 17:00:00,70.855556,66.683333,71.086111,67.093023,58.500000,51.849498,40.50
...,...,...,...,...,...,...,...,...
254,2020-09-20 20:00:00,2.027778,10.175000,2.716667,0.658263,5.350000,2.208333,6.95
255,2020-09-20 21:00:00,4.111111,6.230556,6.280556,1.814085,6.041667,1.187500,6.75
256,2020-09-20 22:00:00,5.244444,6.625000,6.850000,2.311798,6.800000,3.387500,7.45
257,2020-09-20 23:00:00,12.188889,7.733333,9.194444,4.151261,10.241667,3.383333,9.45


In [5]:
df_in1 = df_in[["PT DateTime","Beta-12", "Beta-11","Beta-14", "Beta-18", "Beta-07", "Beta-06"]]
df_test2 = pd.merge(df_in1, df_pub1, on="PT DateTime").dropna()
X_in = df_test2[["Beta-12", "Beta-11","Beta-14", "Beta-18", "Beta-07", "Beta-06"]].to_numpy().flatten(order="F")
Y_in = np.tile(df_test["Reference Average"].to_numpy(), 6)
Y_meb2 = df_test2["Reference Average"].to_numpy()

In [6]:
X = df_test[["Beta-08", "Beta-16", "Beta-17", "Beta-03", "Breakout-08", "Breakout-06"]].to_numpy().flatten(order="F")
Y = np.tile(df_test["Reference Average"].to_numpy(), 6)
X_meb = df_test["Beta-08"].to_numpy()
Y_meb = df_test["Reference Average"].to_numpy()

In [10]:
np.sqrt(calc_error(Y_meb, X_meb))

18.4746922073049

In [ ]:
# linear regression cross validation
error_total_no_off, r_sqaured_total_no_off = cross_val(X, Y, 1)
error_total_off, r_squared_total_off = cross_val(X, Y, 1, off=True)
error_meb_no_off, r_squared_meb_no_off = cross_val(X_meb, Y_meb, 1)
error_meb_off, r_squared_meb_off = cross_val(X_meb, Y_meb, 1, off=True)

In [8]:
## train with MEB, test with other locations
a_meb = linear_fit(X_meb, Y_meb)
X_meb_poly = poly_features(X_meb, 2)
a_meb_poly = linear_fit(X_meb_poly, Y_meb)

print(calc_error(Y_meb, X_meb.dot(a_meb)))
for device in ["Beta-03","Beta-16", "Breakout-06", "Beta-17", "Breakout-08", "Beta-08"]:
    X_meb1 = df_test[device].to_numpy()
    print("Linear R squared: ", r_squared(Y_meb, X_meb1))
    print("Linear MSE: ", np.sqrt(calc_error(Y_meb, X_meb1)))
print("----------------------")
for device in ["Beta-03","Beta-16", "Breakout-06", "Beta-17", "Breakout-08", "Beta-08"]:
    X_meb1 = df_test[device].to_numpy()
    # X_meb_poly1 = poly_features(df_test[device].to_numpy(), 2)
    Y_hat_meb = X_meb1.dot(a_meb)
    # Y_hat_meb_poly = X_meb_poly1.dot(a_meb_poly)
    # print(device + " Linear BIC: ", bic(Y_meb, Y_hat_meb, 1))
    # print(device + " Poly BIC: ", bic(Y_meb, Y_hat_meb_poly, 2))
    print("Linear R squared: ", r_squared(Y_meb, Y_hat_meb))
    print("Linear MSE: ", np.sqrt(calc_error(Y_meb, Y_hat_meb)))
    # print("Poly R squared: ", r_squared(Y_meb, Y_hat_meb_poly))
    # print("Poly MSE: ", calc_error(Y_meb, Y_hat_meb_poly))
# print("----------------------")
# for device in ["Beta-12", "Beta-11","Beta-14", "Beta-18", "Beta-07", "Beta-06"]:
#     X_meb3 = df_test2[device].to_numpy()
#     Y_hat_meb2 = X_meb3.dot(a_meb)
#     print("Linear R squared: ", r_squared(Y_meb2, Y_hat_meb2))
print(a_meb)

206.0715362675306
Linear R squared:  0.9040911517970232
Linear MSE:  19.017699750257478
Linear R squared:  0.8377039567922299
Linear MSE:  24.73902700653241
Linear R squared:  0.8910743486929965
Linear MSE:  20.26719898702703
Linear R squared:  0.881066688135622
Linear MSE:  21.17777866614945
Linear R squared:  0.9171922742796577
Linear MSE:  17.671120051235846
Linear R squared:  0.9094898831062698
Linear MSE:  18.4746922073049
----------------------
Linear R squared:  0.9247898507265576
Linear MSE:  16.84096115445021
Linear R squared:  0.9181718946184481
Linear MSE:  17.566283871639506
Linear R squared:  0.8995080300619633
Linear MSE:  19.46678931678445
Linear R squared:  0.925765665054037
Linear MSE:  16.731352912824306
Linear R squared:  0.9372922062646438
Linear MSE:  15.377630077682351
Linear R squared:  0.9453537063913945
Linear MSE:  14.35519196205786
0.9038691309602482


In [17]:
print(np.sqrt(calc_error(X_meb, Y_meb)))

18.4746922073049


In [13]:
print(r_squared(X_meb, Y_meb))
print(r_squared(X_meb.dot(a_meb), Y_meb))

0.922228226420406
0.9425255956033542


In [27]:
df_saad = df_out[["PT DateTime", "Beta-01"]]
df_saad = pd.merge(df_saad, df_pub1, how="left", on="PT DateTime")
df_saad = df_saad.dropna()
Y_meb3 = df_saad["Reference Average"].to_numpy()
X_meb2 = df_saad["Beta-01"].to_numpy()
Y_hat_meb2 = X_meb2.dot(a_meb)
print("Linear R squared: ", r_squared(Y_meb3, Y_hat_meb2))
print("Linear MSE: ", calc_error(Y_meb3, Y_hat_meb2))

Linear R squared:  0.8861150568622362
Linear MSE:  446.66761505960994


In [11]:
%matplotlib qt
fig = plt.figure(figsize=(15,15))
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.scatter(Y_meb, X_meb, label="original")
plt.scatter(Y_meb, X_meb.dot(a_meb), label="calibrated")
# plt.scatter(X_meb.dot(a_meb), X_meb, label="Regression Line")
plt.plot(X_meb, X_meb, "g")
plt.xlabel("Reference PM2.5 Concentration (µg/$m^{3}$)", fontsize=30)
plt.ylabel("Calibrated data (µg/$m^{3}$)", fontsize=30)
plt.legend(prop={'size': 30})
plt.show()

In [ ]:
%matplotlib qt
names = ["MEB", "Igor", "Stephanie", "Alex", "Edmund", "Brad"]
for i, device in enumerate(["Beta-08", "Beta-16", "Beta-17", "Beta-03", "Breakout-08", "Breakout-06"]):
    fig = plt.figure(figsize=(7,4))
    plt.rc('xtick', labelsize=20)
    plt.rc('ytick', labelsize=20)
    plt.plot(df_out["PT DateTime"], df_out[device], label="Original", color="blue")
    # plt.plot(df_out["PT DateTime"], a_total*df_out[device], label="Corrected", color="red")
    plt.plot(df_out["PT DateTime"], poly_features(df_out[device].to_numpy(),2).dot(a_total_poly), label="Corrected", color="red")
    plt.plot(df_test["PT DateTime"], df_test["Reference Average"], label="Ref Average", color="green")
    plt.xlabel("Date", fontsize=30)
    plt.ylabel("PM2.5 Concentration", fontsize=30)
    plt.title(names[i], fontsize=30)
    if device == "Beta-08":
        plt.legend(prop={'size': 20})
    dtfmt = mdates.DateFormatter("%m-%d")
    plt.gca().xaxis.set_major_formatter(dtfmt)
    plt.show()

In [ ]:
## poly regression
%matplotlib qt
min_error_indices = []
for i in range(30):
    errors = []
    for degree in range(1, 11):
        errors.append(cross_val(X, Y, degree, folds=10))
    min_error_indices.append(errors.index(min(errors)))
#     plt.plot(xs, errors)
# plt.show()
# print(min(errors), errors.index(min(errors)))


In [ ]:
errors = []
for degree in range(1, 11):
    errors.append(cross_val(X, Y, degree, folds=10))

In [ ]:
%matplotlib qt
xs = [i+1 for i in range(len(errors))]
plt.plot(xs, errors)
plt.xlabel("degree", fontsize=30)
plt.ylabel("error", fontsize=30)